In [7]:
import pandas as pd
import sys
import numpy as np
sys.path.append("../src")

In [8]:
M = pd.read_csv("../temp/G_Transit_GeoLife_4thRing.csv", header=None)

In [9]:
M.shape

(2500, 2500)

In [10]:
import mechanism
import trajectory_processor

In [11]:
path_trans_mat = "../temp/TransMat_4thRing.txt"
query = np.loadtxt("../temp/locQuery.txt")

tp = trajectory_processor.TrajectoryProcessor()

tp.load(path_trans_mat, query)

In [12]:
M = pd.read_csv("../temp/TestTrajSet_4thRing.txt", header=None)

In [13]:
trans_mat = np.loadtxt("../temp/TransMat_4thRing.txt")
query = np.loadtxt("../temp/locQuery.txt")

In [24]:
query = np.loadtxt("../temp/locQuery.txt")

In [10]:
M

NameError: name 'M' is not defined

In [5]:
!pip install joblib

  Using cached https://files.pythonhosted.org/packages/28/5c/cf6a2b65a321c4a209efcdf64c2689efae2cb62661f8f6f4bb28547cf1bf/joblib-0.14.1-py2.py3-none-any.whl
You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import sys
sys.path.append("../src")

import pandas as pd
import trajectory_processor
import numpy as np
import matplotlib.pyplot as plt
import util
#from scipy.spatial import ConvexHull, convex_hull_plot_2d
import mechanism

def perturb_trajectory(traj, traj_processor, delta, epsilon, mec, iter_num=20):
    reports = {"trajectories": [], "distance": []}
    
    for _ in range(iter_num):
        
        prior_distribution = np.zeros(traj_processor.size)
        prior_distribution[traj[0]] = 1
        perturbed_trajectory = np.zeros((len(traj) ,2))
        distances = []

        for i, traj in enumerate(traj):
            #print("true state:", np.where(oh_loc==1)[0][0])

            pos_dist = traj_processor.compute_posterior_distribution(prior_distribution)
            state_nos = traj_processor.compute_possible_set_with_graph(pos_dist, None)

            coords = traj_processor.state2loc(state_nos)
            true_loc = traj_processor.state2loc(np.array([traj]))[0]

            mec.load(coords, state_nos)
            mec.build_distribution(epsilon)
            perturbed_loc = mec.perturb(true_loc)
            perturbed_trajectory[i,:] = perturbed_loc

            #print("true:", true_loc, "perturbed_loc", perturbed_loc, "\n")
            prior_distribution = mec.inference(pos_dist, perturbed_loc)

            distances.append(np.linalg.norm(true_loc - perturbed_loc))
            
        reports["trajectories"].append(perturbed_trajectory)
        reports["distance"].append(distances)
        
        
    return reports

In [2]:
query = np.loadtxt("../temp/locQuery.txt")

path_trans_mat = "../temp/TransMat_4thRing.txt" 
data = "../temp/testTrajSet_4thRing.txt"
trajs = pd.read_csv("../temp/testTrajSet_4thRing.txt", header=None).to_numpy()

epsilon = 1
delta = 1e-2

traj_processor = trajectory_processor.TrajectoryProcessor()
traj_processor.load(path_trans_mat, query)

In [ ]:
np.random.seed(0)
im = mechanism.PlanarIsotropicMechanism()
im_reports = perturb_trajectory(trajs[0], traj_processor, delta, epsilon, im, iter_num=1)

../src/mechanism.py:174: RuntimeWarning: invalid value encountered in true_divide
  ks = (vec / self.transformed_vertices)
../src/mechanism.py:190: RuntimeWarning: invalid value encountered in double_scalars
  b = (v2x-(x/y)*v2y)/((x/y)*(v1y-v2y)-v1x+v2x)
../src/mechanism.py:190: RuntimeWarning: divide by zero encountered in double_scalars
  b = (v2x-(x/y)*v2y)/((x/y)*(v1y-v2y)-v1x+v2x)


In [4]:
trajs = pd.read_csv("../temp/testTrajSet_4thRing.txt", header=None).to_numpy()

In [30]:
trajs

array([[1424, 1424, 1424, ..., 1568, 1568, 1568],
       [ 980,  980,  980, ...,  831,  831,  831],
       [ 787,  787,  787, ...,  835,  835,  835],
       ...,
       [1868, 1868, 1868, ..., 1663, 1663, 1663],
       [ 319,  319,  319, ...,  501,  501,  501],
       [2481, 2482, 2482, ..., 2474, 2474, 2474]])

In [6]:
import os
os.path.dirname()

TypeError: dirname() missing 1 required positional argument: 'p'